In [3]:
import os


In [4]:
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\DataScienceProject-1\\research'

In [5]:
os.chdir("../")
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\DataScienceProject-1'

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.DataScience.constants import *
from src.DataScience.utils.common import *
import zipfile
import os
import urllib.request as request
from src.DataScience import logger


In [17]:

class ConfigurationManager():
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config


In [18]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-07 05:01:57,172: INFO: common: yaml file config\config.yaml loaded successfully:]
[2025-02-07 05:01:57,174: INFO: common: yaml file params.yaml loaded successfully:]
[2025-02-07 05:01:57,175: INFO: common: yaml file schema.yaml loaded successfully:]
[2025-02-07 05:01:57,176: INFO: common:  directory artifacts created:]
[2025-02-07 05:01:57,177: INFO: common:  directory artifacts/data_ingestion created:]
[2025-02-07 05:01:59,081: INFO: 1014596764: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4922:2C81D6:117E99:17E31B:67A5466E
Accept-Ranges: bytes
Date: Thu, 06 Feb